#                          Mustafa Can Coşkun - IE 203 - Homework 2

* In this homework, I will provide two distinct ways of obtaining steady state distribution of Ergodic Markov Chain.

1. Matrix Multiplication Method 
2. Monte Carlo Random Walk Simulation

Let me explain formulation of these methods.

### Matrix Multiplication Method

In Matrix Multiplication Method, I am multiplying the Transition Probability Matrix by itself until each column has the same value with the error bound of epsilon. Error bound is given explicitly. 
In order to find the value that will be compared with epsilon, I am choosing one column randomly with "random.randint()" method and taking the average of this column. Then, again, I pick a random row from chosen column and subtracting it from the average value. If the absolute value of this difference is smaller than the epsilon, I break the loop. If not, the while loop iterates up until the difference is smaller than the epsilon. 

### Monte Carlo Random Walk Simulation

Before starting this method, we need an external data set to keep record of the passed states. After defining it, we pick a random row, which is also initial state to start with by "random.randint()". Let's call it X_i. After picking an initial row, we randomly select a number between 0 and 1 in order to select the subsequent step to go, let's call it p. Then for every entry of row, we check if the p is between cumulative sum of entry's up until to p, not including p, and including p. When we find the true interval, our new X_i becomes the index of this true interval and we increase the number of pass times of specified state in our external data set. We repeat this procedure with explicitly specified number of steps. At the end, this function returns the initially defined data set with all the entries are divided with number of steps.

### Absorbing State Included

After calculating the steady state distribution of the markov chain with procedures given above, absorbing row is replaced in the place of a randomly selected state which can be observed in "absorbing_includer()" method. This row is obtained by choosing a random state and creating an array of all zeros except the index of randomly selected number. Then all process is repeated with above procedures.


## Now Let's Go to Python Implementation of This Process

### Importing libraries of numpy, random, time and IPython.

In [2]:
import numpy as np
import pandas as pd
import random
import time
from IPython.display import display

### Creating the Transition Probability Matrix and implementing the multplication and Monte Carlo Simulation method.

In [7]:
class TPM_Matrix(): #Class Representing Transition Probability Matrix and Related Operations

    def __init__(self,size,epsilon = 0.000005, step_size = 200000):
        self._size = size 
        self._matrix  = [] #Probability Transition Matrix
        self._MC_Visits = {f"State_{i}": 0 for i in  range(self._size)} #Keeping Record of Number of Visits in Monte Carlo Method
        self._epsilon = epsilon
        self._step_size = step_size

    def generate_matrix(self): #Generating Ergodic Markov Chain Matrix
        for i in range(self._size):
            a = [random.random() for _ in range(self._size)]
            sum_a = sum(a)
            b = list()
            for j in range(len(a)): #This loop is created in order all entrys in a row to sum up to 1.
                b.append(a[j]/sum_a) 
            self._matrix.append(np.array(b))
        self._matrix = np.array(self._matrix)

    def result_monitoring(self): #Monitoring the results of steady state operations.
        df1 = pd.DataFrame(self.matrix_multp(),columns = ["Steady State by Multp."])
        df2 = pd.DataFrame(self.mc_simulation(),columns = ["Steady State by MC"])
        df_merged = df1.merge(df2,left_index = True,right_index = True)
        df_merged["Diff."] = abs(df_merged["Steady State by Multp."] - df_merged["Steady State by MC"])
        mean_diff  = df_merged["Diff."].mean()
        print(f"*********************************************** Matrix Size = {self._size} ***********************************************")
        display(df_merged)
        print(f"Mean Numerical Difference = {mean_diff}")
    
    def matrix_multp(self):
        result = self._matrix

        while True:
            X_i = random.randint(0,self._size-1) #Choosing random state
            X_j = random.randint(0,self._size-1) #Choosing random entry for comparison

            difference = abs(np.array([result[i][X_i] for i in range(self._size)]).mean() - result[X_j][X_i])
            result = np.matmul(result,result)

            if difference < self._epsilon:
                mean_result = np.array([np.array([result[j][i] for j in range(len(result)) ]).mean() for i in range(len(result))])
                #print(result)
                return mean_result
                break

    def mc_simulation(self): #Monte-Carlo Simulation 

        X_i = random.randint(0,self._size-1) #In order to obtain initial state.

        for _ in range(self._step_size):
            j = np.random.random()
            for x in range(len(self._matrix[X_i])):
                if self._matrix[X_i][:x].sum() < j <= self._matrix[X_i][:x+1].sum():
                    X_i = x
                    self._MC_Visits[f"State_{x}"] += 1
                    break
        
        return np.array([j / self._step_size for j in self._MC_Visits.values()])

    def absorbing_includer(self):
        self._MC_Visits = {f"State_{i}": 0 for i in  range(self._size)}
        X_i = random.randint(0,self._size - 1) #Randomly choosing absorbing state
        absorbing_row = np.array([0]* self._size)
        absorbing_row[X_i] = 1
        self._matrix[X_i] = absorbing_row
        print("******************************************** Absorbing Included ***********************************************")
        self.result_monitoring()

### Now, Let's create the objects of TPM_Matrix class with size of 5,25 and 50 and observe the calculations.

In [8]:
Matrix_5 = TPM_Matrix(5)
Matrix_25 = TPM_Matrix(25)
Matrix_50 = TPM_Matrix(50)

Matrices = [Matrix_5,Matrix_25,Matrix_50]

for i in Matrices:
    i.generate_matrix()
    i.result_monitoring()
    i.absorbing_includer()

*********************************************** Matrix Size = 5 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.195512,0.195605,0.000093
1,0.232221,0.231930,0.000291
2,0.171119,0.169620,0.001499
3,0.177256,0.178315,0.001059
4,0.223891,0.224530,0.000639


Mean Numerical Difference = 0.000716046683456667
******************************************** Absorbing Included ***********************************************
*********************************************** Matrix Size = 5 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,1.274834e-09,0.0,1.274834e-09
1,1.000000e+00,1.0,4.117174e-09
2,8.970522e-10,0.0,8.970522e-10
3,1.172933e-09,0.0,1.172933e-09
4,7.723553e-10,0.0,7.723553e-10


Mean Numerical Difference = 1.6468695893375964e-09
*********************************************** Matrix Size = 25 ***********************************************


...

Mean Numerical Difference = 4.846804880590703e-05
*********************************************** Matrix Size = 50 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.021214,0.021215,0.000001
1,0.019793,0.019780,0.000013
2,0.018361,0.019015,0.000654
3,0.019770,0.020175,0.000405
4,0.016462,0.016220,0.000242
5,0.019663,0.019370,0.000293
6,0.020885,0.020650,0.000235
7,0.019631,0.019890,0.000259
8,0.020103,0.020985,0.000882
9,0.020831,0.020205,0.000626


Mean Numerical Difference = 0.0003190165845028336
******************************************** Absorbing Included ***********************************************
*********************************************** Matrix Size = 50 ***********************************************


,Steady State by Multp.,Steady State by MC,Diff.
0,0.000001,0.000000,0.000001
1,0.000001,0.000010,0.000009
2,0.000001,0.000000,0.000001
3,0.000001,0.000000,0.000001
4,0.000001,0.000010,0.000009
5,0.000001,0.000010,0.000009
6,0.000001,0.000005,0.000004
7,0.000001,0.000010,0.000009
8,0.000001,0.000015,0.000014
9,0.000001,0.000010,0.000009


Mean Numerical Difference = 1.42907072389705e-05


### After observing the numerical approximations and differences, it will be beneficial to observe the efficiency of methods in terms of processing time.

In [14]:
size_5 = TPM_Matrix(5)
size_25 = TPM_Matrix(25)
size_50 = TPM_Matrix(50)

matrix_array = [size_5,size_25,size_50]
time_multp = []
time_simul = []

for i in matrix_array:
    i.generate_matrix()
    start = time.time()
    i.matrix_multp()
    intermediary = time.time()
    i.mc_simulation()
    end = time.time()
    
    matrix_time = intermediary - start
    simul_time = end - intermediary
    
    time_multp.append(matrix_time)
    time_simul.append(simul_time)

time_multp = np.array(time_multp)
time_simul = np.array(time_simul)


df1 = pd.DataFrame(time_multp, columns = ["Matrix Multp."])
df2 = pd.DataFrame(time_simul, columns = ["Monte Carlo Simul"])
df_merged = df1.merge(df2,left_index = True, right_index = True)
df_merged["Diff"] = df2["Monte Carlo Simul"] - df1["Matrix Multp."]
display(df_merged)

,Matrix Multp.,Monte Carlo Simul,Diff
0,0.000790,2.709408,2.708618
1,0.000560,10.357597,10.357037
2,0.002127,20.471201,20.469074
